In [57]:
import json
import os
import random
import time

In [58]:
import h3
import geopandas as gpd
import pandas as pd

In [59]:
from kroger_api import KrogerAPI
from kroger_api.utils.env import load_and_validate_env, get_zip_code
from shapely.geometry import Point
from tqdm.notebook import tqdm

## Get State Data

In [60]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

In [61]:
states_df = states_df[["STUSPS", "NAME", "geometry"]]

## Get Population data

In [62]:
state_populations = pd.read_excel(
    "data/NST-EST2024-POP.xlsx", sheet_name=None, engine="openpyxl"
)

In [63]:
state_populations_df = state_populations["NST-EST2024-POP"][
    [
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)",
        "Unnamed: 5",
    ]
]
state_populations_df = state_populations_df.rename(
    columns={
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)": "NAME",
        "Unnamed: 5": "POPULATION",
    }
)
state_populations_df["NAME"] = state_populations_df["NAME"].str[1:]

In [64]:
states_with_population_df = states_df.merge(state_populations_df, on="NAME", how="left")
states_with_population_df = states_with_population_df[
    ["STUSPS", "NAME", "POPULATION", "geometry"]
]

## Get Store Data

In [9]:
load_and_validate_env(["KROGER_CLIENT_ID", "KROGER_CLIENT_SECRET"])
zip_code = get_zip_code(default="10001")

In [10]:
kroger = KrogerAPI()
token_info = kroger.authorization.get_token_with_client_credentials("product.compact")

Found saved token, will test if it's still valid...
Token appears invalid, requesting a new one


In [11]:
res = 3
distance = 0.5

usa_geom = states_df[~states_df["STUSPS"].isin(["PR", "VI", "GU", "MP", "AS"])][
    "geometry"
].union_all()

h3_shape = h3.geo_to_h3shape(usa_geom.buffer(distance=distance))
h3_cells = h3.h3shape_to_cells(h3_shape, res=res)
geojson = h3.cells_to_geo(h3_cells)

In [12]:
stores = []
for cell in tqdm(h3_cells, desc="Parsing Stores"):
    lat, lng = h3.cell_to_latlng(cell)
    locations = kroger.location.search_locations(
        lat=lat, lon=lng, radius_in_miles=100, limit=200
    )
    if len(locations["data"]):
        for location in locations["data"]:
            location["geometry"] = Point(
                location["geolocation"]["longitude"],
                location["geolocation"]["latitude"],
            )
            location["STUSPS"] = location["address"]["state"]
            stores.append(location)
    else:
        continue
    time.sleep(random.uniform(0.01, 0.5))

Parsing Stores:   0%|          | 0/911 [00:00<?, ?it/s]

In [45]:
stores_gdf = gpd.GeoDataFrame(stores, crs=4326)
stores_gdf = stores_gdf.drop_duplicates("locationId")
stores_gdf.to_file("data/all_stores.gpkg")

In [65]:
stores_gdf = gpd.read_file("data/stores.gpkg")

In [70]:
stores_gdf = stores_gdf[stores_gdf["name"].str.split(" - ").str[0] == "Harris Teeter"]

In [71]:
ht_gdf = stores_gdf[stores_gdf["chain"] == "HART"]
ht_gdf = ht_gdf[~ht_gdf["phone"].isna()].reset_index(drop=True)
ht_gdf.to_file("data/stores.gpkg")

In [72]:
ht_count_df = ht_gdf.groupby("STUSPS").size().reset_index(name="COUNT")

## Combine With States

In [73]:
store_count_gdf = states_with_population_df.merge(ht_count_df, on="STUSPS", how="left")

In [74]:
store_count_gdf = store_count_gdf.dropna()

In [78]:
store_count_gdf["per_100k"] = (
    store_count_gdf["COUNT"] / (store_count_gdf["POPULATION"] / 100_000)
).round(decimals=2)
store_count_gdf["per_1m"] = (
    store_count_gdf["COUNT"] / (store_count_gdf["POPULATION"] / 1_000_000)
).round(decimals=2)

In [79]:
store_count_gdf = store_count_gdf.to_crs(9311)
store_count_gdf.to_file(f"data/Harris_Teeters_Per_State.gpkg")

In [80]:
store_count_gdf

,STUSPS,NAME,POPULATION,geometry,COUNT,per_100k,per_1m
1,NC,North Carolina,10881189.0,"MULTIPOLYGON (((2154808.027 -692253.992, 21554...",151.0,1.39,13.88
3,VA,Virginia,8734685.0,"MULTIPOLYGON (((2098153.247 -490400.387, 20981...",53.0,0.61,6.07
9,FL,Florida,22904868.0,"MULTIPOLYGON (((1997726.172 -1929592.915, 1997...",1.0,0.00,0.04
18,GA,Georgia,11064432.0,"MULTIPOLYGON (((1774018.899 -1324581.179, 1774...",1.0,0.01,0.09
33,SC,South Carolina,5387830.0,"MULTIPOLYGON (((1898434.953 -1099843.186, 1898...",30.0,0.56,5.57
36,DC,District of Columbia,687324.0,"POLYGON ((1950799.304 -402452.215, 1951231.574...",3.0,0.44,4.36
40,MD,Maryland,6217062.0,"MULTIPOLYGON (((2067172.995 -478871.787, 20670...",18.0,0.29,2.90
43,DE,Delaware,1036423.0,"MULTIPOLYGON (((2060773.554 -301785.14, 206090...",2.0,0.19,1.93
